# MA22018 Coursework 2

In [1]:
import numpy as np

def syntheticY(n):
    """
    Synthetic rank-2 n x n matrix
    """
    U = np.vstack((np.arange(n)/n, (1-np.arange(n)/n)**2)).T
    W = np.vstack((np.arange(n)/n, (1-np.arange(n)/n)**2)).T
    return U @ W.T

def bernoulli(mu,n,p,B):
    """
    Bernoulli sampling with the number of samples divisible by B.
    Produces a set of the first m index pairs I = {(i1,i2)} where
    i1 = 0,...,mu-1, i2 = 0,...,n-1, each pair 
    is sampled with probability p, and B divides m.
    """
    I = []
    for i1 in range(mu):
        for i2 in range(n):
            if np.random.rand()<p:
                I.append([i1,i2])
    return np.array(I[:B*(len(I)//B)])

def Loss(U,W,Y,I):
    """
    Loss function (2), where 
    U, W are n x r and mu x r factor matrices,
    Y is a mu x n data matrix, 
    and I is a m x 2 matrix of index pairs
    """
    L = 0
    for i1,i2 in I:
        L = L + (U[i1] @ W[i2] - Y[i1,i2])**2
    return L / I.shape[0]

def initial(mu,n,r):
    """
    Initial guess of the matrix factors
    """
    U0 = np.vstack((np.identity(r), np.zeros((mu-r,r))))
    W0 = np.vstack((np.identity(r), np.zeros((n-r,r))))
    return U0, W0


def test_pointwise_gradient_1():
    """
    Unit tests for the pointwise gradient
    """
    Y = syntheticY(4)
    U,W = initial(4,4,2)
    vu = np.array([0,-1.125])
    vw = np.array([-1.125,0])
    assert np.allclose(pointwise_gradient(U,W,Y,0,1)[0], vu), "wrong vu in test 1"
    assert np.allclose(pointwise_gradient(U,W,Y,0,1)[1], vw), "wrong vw in test 1"
    
def test_pointwise_gradient_2():
    Y = syntheticY(4)
    U,W = initial(4,4,2)
    vu = np.array([0,0])
    vw = np.array([-0.5,0])
    assert np.allclose(pointwise_gradient(U,W,Y,0,2)[0], vu), "wrong vu in test 2"
    assert np.allclose(pointwise_gradient(U,W,Y,0,2)[1], vw), "wrong vw in test 2"
    
def test_pointwise_gradient_3():
    Y = syntheticY(4)[:, 1:]
    U,W = initial(4,3,2)    
    vu = np.array([0,-0.625])
    vw = np.array([0,0])
    assert np.allclose(pointwise_gradient(U,W,Y,2,1)[0], vu), "wrong vu in test 3"
    assert np.allclose(pointwise_gradient(U,W,Y,2,1)[1], vw), "wrong vw in test 3"
    
    
def test_full_gradient_1():
    """
    Unit tests for the full gradient
    """
    Y = syntheticY(4)
    U,W = initial(4,4,2)
    Gu = np.array([[0,-1.125], [0,     0], [0,0], [0,0]])
    Gw = np.array([[0,     0], [-1.125,0], [0,0], [0,0]])
    assert np.allclose(full_gradient(U,W,Y, np.array([[0,1]]))[0], Gu), "wrong Gu in test 1"
    assert np.allclose(full_gradient(U,W,Y, np.array([[0,1]]))[1], Gw), "wrong Gw in test 1"
    
def test_full_gradient_2():
    Y = syntheticY(4)
    U,W = initial(4,4,2)
    Gu = np.array([[0,    0], [-9/16,0], [0,   0], [0,0]]) 
    Gw = np.array([[0,-9/16], [0,    0], [-1/4,0], [0,0]])
    assert np.allclose(full_gradient(U,W,Y, np.array([[1,0], [0,2]]))[0], Gu), "wrong Gu in test 2"
    assert np.allclose(full_gradient(U,W,Y, np.array([[1,0], [0,2]]))[1], Gw), "wrong Gw in test 2"
    
def test_full_gradient_3():
    Y = syntheticY(4)[:, 1:]
    U,W = initial(4,3,2)
    Gu = np.array([[0,-1/6], [-97/384,0], [0,-5/24], [0,0]]) 
    Gw = np.array([[0,-97/384], [-1/6,0], [0,   0]])
    assert np.allclose(full_gradient(U,W,Y, np.array([[1,0], [0,1], [2,1]]))[0], Gu), "wrong Gu in test 3"
    assert np.allclose(full_gradient(U,W,Y, np.array([[1,0], [0,1], [2,1]]))[1], Gw), "wrong Gw in test 3"